### 31. 인자에 대해 이터레이션할 때는 방어적이 돼라
* 이터레이터가 결과를 단 한번만 만들어내기 때문에 주의해야 한다

In [ ]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)
it = read_visits('my_numbers.txt')
print(list(it))
print(list(it))   # 이미 모든 원소를 다 소진

In [ ]:
# 이터레이터 프로토콜을 구현한 컨테이너 클래스 사용
class ReadVisit:
    def __init__(self, data_path):
        self.data_path = data_path

    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

visits = ReadVisits(path)

print(list(visits))
print(list(visits))

* 호출할 때마다 새로운 이터레이터를 반환하여 메모리를 줄일 수 있다
* __iter __ 메서드를 제너레이터로 정의하면 쉽게 이터레이터 컨테이너 타입을 정의할 수 있다

### 32. 긴 리스트 컴프리헨션보다는 제너레이터 식을 사용하라
* 리스트 컴프리헨션은 메모리를 상당히 많이 사용한다는 단점이 있음 -> 프로그램 중단 가능
* 따라서 **입력이 큰 경우에는 제너레이터 식 사용**하기 -> 매우 빠르게 실행, 메모리도 효율적이다

제너레이터 식 : 리스트 컴프리헨션과 제너레이터를 일반화한 것

In [ ]:
# 이렇게
it = (len(x) for x in open('my_file.txt'))

* 메모리 사용량이 많아질수록 리스트 컴프리헨션은 문제를 일으킬 수 있다
* 제너레이터 식은 이터레이터처럼 한 번에 원소 하나씩 출력 -> 메모리 문제를 피할 수 있다


### 33. yield from을 사용해 여러 제너레이터를 합성하라
* 반복적 코드 줄이기

In [ ]:
def move(period, speed):
    for _ in range(period):
        yield speed


def pause(delay):
    for _ in range(delay):
        yield 0


def animate():
    for delta in move(4, 5.0):
        yield delta
    for delta in pause(3):
        yield delta
    for delta in move(2, 3.0):
        yield delta

# yield from 사용
def animate():
    yield from move(4, 5.0)
    yield from pause(3)
    yield from move(2, 3.0)


* yield from -> 여러 내장 제너레이터를 모아서 하나로 합성
* 각각의 제너레이터를 이터레이션하면서 출력하는 것 보다 성능이 좋다!

### 34. send로 제너레이터에 데이터를 주입하지 말라
* for루프나 next내장 함수로 제너레이터를 이터레이션 하지 않고 send메서드를 호출하면 None값을 뱉는 경우 발생   
* ??????????

In [ ]:
it = iter(my_gen())
output = it.send(None)
print(output)

try:
    it.send("Hi")
except StopIteration:
    pass

### 35. 제너레이터 안에서 throw 상태를 변화시키지 말라
* 제너레이터의 고급 기능 yield from과 send 메서드
* 이 외에도 제너레이터 안에서 Exception을 다시 던질 수 있는 throw메서드가 있다.

**작동 방식**
어떤 제너레이터에 대해 throw 호출 -> 이 제너레이터는 값을 내놓은 yield로부터 평소처럼 제너레이터 실행을 계속하는 대신 throw가 제공한 Exception을 다시 던짐
* 이 기능은 제너레이터, 제너레이터를 호출하는 쪽 사이 양방향 통신 수단을 제공함
* 근데 throw는 되도록 사용하지 말고,, iter메소드를 구현해서 처리하도록 하자

In [ ]:
# Reset 예외가 발생할 때마다 period 재설정
class Reset(Exception):
    pass

def timer(period):
    current = period
    while current:
        current -= 1
        try:
            yield current
        except Reset:
            current = period

# 이터러블 클래스를 사용해서 예외처리하기
class Timer:
    def __init__(self, period):
        self.current = period
        self.period = period

    def reset(self):
        self.current = self.period

    def __iter__:
        while self.current:
            self.current -= 1
            yield self.current

### 36. 이터레이터나 제너레이터를 다룰 때는 itertools를 사용하라
* 복잡한 이터레이션 코드를 작성할 때 쓸만한 기능이 많다

### 37. 내장 타입을 여러 단계로 내포시키기보다는 클래스를 합성하라


In [ ]:
class SimpleGradeBook:
    def __init__(self):
        self._grades = {}

    def add_student(self, name):
        self._grades[name] = []

    def report_grade(self, name, score):
        self._grades[name].append(score)

    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

# 이런식으로 로직이 추가돼서 너무 복잡해진다면
# 더이상 내장 타입을 사용하지 말고 클래스 계층 구조를 사용해야 한다
class WeightedGradeBook:
    def __init__(self):
        self._grades = {}

    def add_student(self, name):
        self._grades[name] = defaultdict(list)

    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list =  by_subject[subject]
        grade_list.append((score, weight))

    def average_grade(self, name, subject):
        by_subject = self._grades[name]
        score_sum = score_count = 0
        for subject, score in by_subject.items():
            subject_avg = total_weight = 0
            for score , weight in subject:
                subject_avg += score * weight
                total_weight += weight
            score_sum += subject_avg / total_weight
            score_count += 1
        return score_sum / score_count

# 클래스를 활용한 리팩토링
grades = []
grades.append((95. 0.45))
grades.append((85. 0.55))
total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)

In [ ]:
# 코드는 길어졌지만 더 읽기 쉽고 확장성이 좋음
class Subject:
    def __init__(self):
        self._grades = []

    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))

    def average_grade(self):
        total = total_weight = 0
        for grade in self._grades: # grade: namedtuple
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject)

    def get_subject(self, name):
        return self._subjects[name]

    def average_grade(self):
        total = count = 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

class GradeBook:
    def __init__(self):
        self._students = defaultdict(Student)

    def get_student(self, name):
        return self._students[name]

### 38. 간단한 인터페이스의 경우 클래스 대신 함수를 받아라
* "훅" : 함수를 전달해서 동작을 원하는 대로 바꿀 수 있게 해주는 내부 API


### 기타 내용들

In [ ]:
# sympy 활용하기
from sympy import *
x = symbols('x')
a = Integral(cos(x)*exp(x), x)
a
Eq(a, a.doit())

Eq(Integral(exp(x)*cos(x), x), exp(x)*sin(x)/2 + exp(x)*cos(x)/2)